Домашнее задание.
1. Попробуйте обучить нейронную сеть с применением одномерных сверток для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech
2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
max_words = 2000
max_len = 20
num_classes = 1

epochs = 30
batch_size = 512
print_batch_n = 100

In [3]:
df_train = pd.read_csv('twitter_data/train.csv')
df_test = pd.read_csv('twitter_data/test.csv')

In [4]:
df_train, df_val = train_test_split(df_train, test_size=0.3, shuffle=True)

In [5]:
!pip install stop-words pymorphy2

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [7]:
sw = set(get_stop_words("en"))

In [8]:
puncts = set(punctuation)

In [9]:
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
#     txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [10]:
from tqdm import tqdm
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)
df_test['tweet'] = df_test['tweet'].progress_apply(preprocess_text)

100%|███████████████████████████████████| 17197/17197 [00:02<00:00, 7691.06it/s]


In [11]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()

In [12]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /home/boormistr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['goodbye', 'safety', 'net', 'hello', 'selfemployment']

In [14]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [15]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
len(tokens_filtered_top)

1999

In [16]:
tokens_filtered_top[:10]

['user', 'love', 'day', 'happy', 'amp', 'just', 'will', 'im', 'life', 'time']

In [17]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'user': 1,
 'love': 2,
 'day': 3,
 'happy': 4,
 'amp': 5,
 'just': 6,
 'will': 7,
 'im': 8,
 'life': 9,
 'time': 10,
 'u': 11,
 'like': 12,
 'today': 13,
 'new': 14,
 'now': 15,
 'positive': 16,
 'get': 17,
 'thankful': 18,
 'people': 19,
 'good': 20,
 'cant': 21,
 'bihday': 22,
 'one': 23,
 'see': 24,
 'can': 25,
 'dont': 26,
 'fathers': 27,
 'want': 28,
 'go': 29,
 'smile': 30,
 'healthy': 31,
 'take': 32,
 'got': 33,
 'work': 34,
 'weekend': 35,
 'friday': 36,
 'make': 37,
 'fun': 38,
 'best': 39,
 'summer': 40,
 'us': 41,
 'great': 42,
 'beautiful': 43,
 'family': 44,
 'way': 45,
 '2': 46,
 'bull': 47,
 'friends': 48,
 'need': 49,
 'first': 50,
 'days': 51,
 'music': 52,
 'going': 53,
 'really': 54,
 'world': 55,
 'back': 56,
 'wait': 57,
 'morning': 58,
 'know': 59,
 'sad': 60,
 'tomorrow': 61,
 'orlando': 62,
 'never': 63,
 'week': 64,
 'fathersday': 65,
 'happiness': 66,
 'cute': 67,
 'well': 68,
 'think': 69,
 'feel': 70,
 'much': 71,
 'trump': 72,
 'blog': 73,
 'model': 74,
 

In [18]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return result[-maxlen:] + padding

In [19]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["tweet"]])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["tweet"]])

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim = 128, out_channel = 128,
                 num_classes = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
        self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, out_channel//2)
        self.linear_2 = nn.Linear(out_channel//2, num_classes)
        
    def forward(self, x):        
        output = self.embedding(x)        
        output = output.permute(0, 2, 1)
        output = self.conv_1(output)
        output = self.relu(output)
        output = self.pool(output)
        
        output = self.conv_2(output)
        output = self.relu(output)
        output = self.pool(output)
        
        output = torch.max(output, axis=2).values
        output = self.linear_1(output)
        output = self.relu(output)
        output = self.linear_2(output)
        output = F.sigmoid(output)
        
        return output

In [22]:
from torch.utils.data import DataLoader, Dataset

class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [23]:
df_train.label.sum()

1602

In [24]:
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [25]:
model = Net(vocab_size=max_words)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
print(model)
print('Parameters: ', sum([param.nelement() for param in model.parameters()]))

Net(
  (embedding): Embedding(2000, 128)
  (conv_1): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (linear_1): Linear(in_features=128, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=1, bias=True)
)
Parameters:  346497


In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [28]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []

for epoch in range(epochs):
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        
        loss = loss.item()
        running_items += len(labels)
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
    
    model.eval()
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
          f'Step: [{i + 1}/{len(train_loader)}]. '\
          f'Loss: {loss:.3f}, ' \
          f'Acc: {running_right / running_items:.5f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)
    
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = model(data[0].to(device))
        
        test_loss = criterion(test_outputs, test_labels.float().view(-1,1))
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
        
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss: .3f}. \
            Test acc: {test_running_right / test_running_total: .5f}')
    
    model.train()
    
print('Training is finished!')

/home/boormistr/.local/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/30]. Step: [44/44]. Loss: 0.116, Acc: 0.91025. Test loss:  0.009.             Test acc:  0.93326
Epoch [2/30]. Step: [44/44]. Loss: 0.148, Acc: 0.94408. Test loss:  0.004.             Test acc:  0.94890
Epoch [3/30]. Step: [44/44]. Loss: 0.110, Acc: 0.96335. Test loss:  0.106.             Test acc:  0.95067
Epoch [4/30]. Step: [44/44]. Loss: 0.063, Acc: 0.97720. Test loss:  0.389.             Test acc:  0.94452
Epoch [5/30]. Step: [44/44]. Loss: 0.023, Acc: 0.98512. Test loss:  0.000.             Test acc:  0.94713
Epoch [6/30]. Step: [44/44]. Loss: 0.019, Acc: 0.98918. Test loss:  0.240.             Test acc:  0.94191
Epoch [7/30]. Step: [44/44]. Loss: 0.011, Acc: 0.99146. Test loss:  0.015.             Test acc:  0.94827
Epoch [8/30]. Step: [44/44]. Loss: 0.017, Acc: 0.99303. Test loss:  0.004.             Test acc:  0.94619
Epoch [9/30]. Step: [44/44]. Loss: 0.023, Acc: 0.99294. Test loss:  0.063.             Test acc:  0.93878
Epoch [10/30]. Step: [44/44]. Loss: 0.021, Acc